In [ ]:
import torch
import numpy as np
import os
import json
from EduNLP.Pretrain import train_elmo
from EduNLP.Vector import ElmoModel, T2V
from EduNLP.I2V import Elmo, get_pretrained_i2v

# 训练自己的Elmo模型
## 1. 数据

In [ ]:
BASE_DIR = "E:\Workustc\EduNLP\workMaster\EduNLP"

data_dir = f"{BASE_DIR}/tests/test_vec/test_data"
output_dir = f"{BASE_DIR}/examples/test_model/data/elmo"


def raw_data():
    _data = []
    data_path = os.path.join(data_dir, "OpenLUNA.json")
    with open(data_path, encoding="utf-8") as f:
        for line in f.readlines():
            _data.append(json.loads(line))
    return _data

def stem_data(data):
    _data = []
    tokenizer = ElmoTokenizer()
    for e in data:
        d = tokenizer.tokenize(item=data['stem'], freeze_vocab=False)
        if d is not None:
            _data.append(d)
    assert _data
    return _data

raw_data = raw_data()
train_items = stem_data(raw_data)

BASE_DIR = "E:\Workustc\EduNLP\workMaster\EduNLP"
data_dir = f"{BASE_DIR}/tests/test_vec"
output_dir = f"{BASE_DIR}/examples/test_model/data"

## 2. 训练和评估

In [ ]:
train_params = {
  "emb_dim": 512,
  "hid_dim": 512,
  "batch_size": 2,
  "epochs": 3,
  "lr": 5e-4,
  "device": None
}

train_elmo(train_items, output_dir, **train_params)

## 3.使用模型

In [ ]:
item = {'stem': '如图$\\FigureID{088f15ea-8b7c-11eb-897e-b46bfc50aa29}$, \
          若$x,y$满足约束条件$\\SIFSep$，则$z=x+7 y$的最大值为$\\SIFBlank$'}

tokenizer_kwargs = {"path": os.path.join(output_dir, "vocab.json")}
i2v = Elmo('elmo', 'elmo', output_dir, tokenizer_kwargs=tokenizer_kwargs)

i_vec, t_vec = i2v(item['stem'])
# or
# i_vec = i2v.infer_item_vector(item['stem'])
# t_vec = i2v.infer_token_vector(item['stem'])

print(i_vec.shape) # == torch.Size([x, x])
print(t_vec.shape) # == torch.Size([x, x, x])